<a href="https://colab.research.google.com/github/yullano90/DataDrafts/blob/main/Modelo_ETL_sugestivo_para_agendamento_TXT_SISREG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Siga os passos abaixo, cuidadosamente para evitar erros de execução:
---
- Crie um diretório e descarregue os arquivos do SISREG, dê a ele o nome "TXT";

- O padrão de nome dos arquivos é: "HOSPITAL FULANO DAS QUANTAS-Outubro-2021";




Crie o folder "ETL_Agend_Sisreg", e lá insira as consultas abaixo:

- "1_Dir_Arquivos"

In [ ]:
"Diretório local e/ou servidor de arquivos" 
meta
 [
      IsParameterQuery=true, 
      Type="Any", 
      IsParameterQueryRequired=true
    ]

- "2_Arquivo_Modelo"

In [ ]:
let
    Fonte = Folder.Files(#"1_Dir_Arquivos"),
    Navegação1 = Fonte{0}[Content]
in
    Navegação1

- "3_ETL_Arquivo_Modelo"

In [ ]:
let
    Fonte = (#"4_Texto_Base64" as binary) => let
    Fonte = Csv.Document(#"4_Texto_Base64",[Delimiter=";", Columns=38, Encoding=65001, QuoteStyle=QuoteStyle.None])
in
    Fonte
in
    Fonte

- "4_Texto_Base64"

In [ ]:
#"2_Arquivo_Modelo" 
meta [
        IsParameterQuery=true, 
        BinaryIdentifier=#"2_Arquivo_Modelo", 
        Type="Binary", 
        IsParameterQueryRequired=true
    ]

- "5_ETL_Global"

In [ ]:
let
    Fonte = Csv.Document(
        #"4_Texto_Base64",
        [
            Delimiter=";", 
            Columns=38, 
            Encoding=65001, 
            QuoteStyle=QuoteStyle.None
        ]
    )
in
    Fonte

- "6_Dataset_Final"

In [ ]:

let

    Dir_Arquivos = 
        Folder.Files(
            #"1_Dir_Arquivos"
            ),


        ELT_Auto1 =
            Table.SelectRows(
                Dir_Arquivos, 
                each [Attributes]?[Hidden]? <> true
            ),


    ELT_Auto2 = 
        Table.AddColumn(
            ELT_Auto1, 
            "Transformar Arquivo (3)", 
            each #"3_ETL_Arquivo_Modelo"(
                [Content]
                )
            ),


    ELT_Auto3 = 
        Table.RenameColumns(
            ELT_Auto2, 
            {"Name", "Nome da Origem"}
            ),


    ELT_Auto4 = 
        Table.SelectColumns(
            ELT_Auto3, 
                {
                    "Nome da Origem", 
                    "Transformar Arquivo (3)"
                }
            ),


    Exp_Matrix = 
        Table.ExpandTableColumn(
            ELT_Auto4, 
                    "Transformar Arquivo (3)", 
                    Table.ColumnNames(
                    #"3_ETL_Arquivo_Modelo"(
                    #"2_Arquivo_Modelo"
                )
            )
        ),


    Dvd_C = 
        Table.SplitColumn(
            Exp_Matrix, 
            "Nome da Origem", 
                Splitter.SplitTextByDelimiter("-", 
                    QuoteStyle.Csv), 
                        {
                            "Nome da Origem.1", 
                            "Nome da Origem.2", 
                            "Nome da Origem.3"
                            }
                        ),

    Alter_Type1 = 
        Table.TransformColumnTypes(
            Dvd_C,
            {
                {"Nome da Origem.1", type text}, 
                {"Nome da Origem.2", type text}, 
                {"Nome da Origem.3", type text}
                }
            ),

    Ext_Carac = 
        Table.TransformColumns(
            Alter_Type1, 
            {
                {"Nome da Origem.3", 
                each Text.Start(_, 4), type text}
                }
            ),

    Replaced_Value = 
        Table.ReplaceValue(
            Ext_Carac,".","/",Replacer.ReplaceText,
            {
                "Column7", 
                "Column12", 
                "Column30", 
                "Column32"
                }
            ),

    Alter_Type2 = 
        Table.TransformColumnTypes(
            Replaced_Value,{
                {"Column32", type date}, 
                {"Column30", type date}, 
                {"Column12", type date}, 
                {"Column7", type date}
             }
        ),
        
    Rnm_C = 
        Table.RenameColumns(
            Alter_Type2,
            {
                {"Nome da Origem.1", "Executante"}, 
                {"Nome da Origem.2", "Mês"}, 
                {"Nome da Origem.3", "Ano"}, 
                {"Column1", "Cod_Solicitacao"}, 
                {"Column2", "Cod_Interno"}, 
                {"Column3", "Cod_Unificado"}, 
                {"Column4", "Procedimento"}
            }
        ),

    Filter_R = 
        Table.SelectRows(
            Rnm_C, 
            each ([Column6] <> "")
                ),

    
    Rnm_C_1 = 
        Table.RenameColumns(
            Filter_R,
            {
                {"Column5", "CPF"}, 
                {"Column6", "Profissional"}, 
                {"Column7", "Data_Execução"}, 
                {"Column8", "Horario"}, 
                {"Column9", "Logic"}, 
                {"Column10", "CNS/SUS"}, 
                {"Column11", "Paciente"}, 
                {"Column12", "Data_Nasc"}, 
                {"Column13", "Idade"}, 
                {"Column14", "Lote_Paciente"}, 
                {"Column15", "Nome_Mae"}, 
                {"Column18", "Complemento_2"}, 
                {"Column16", "Endereço"}, 
                {"Column17", "Complemento1"}, 
                {"Column19", "Numero"}, 
                {"Column20", "Bairro"}, 
                {"Column21", "CEP"}, 
                {"Column22", "Telefone"}, 
                {"Column23", "Municipio"}, 
                {"Column24", "Cod_IBGE"}, 
                {"Column25", "Dominio"}
           }
        ),


    Rnm_C2 = 
        Table.RemoveColumns(
            Rnm_C_1,
            {"Column26"}
            ),


    Rnm_C3 = 
        Table.RenameColumns(
            Rnm_C2,
            {
                {"Column27", "CNES"}, 
                {"Column28", "Solicitante"}, 
                {"Column29", "Sexo"}, 
                {"Column31", "Marcador_SISREG"}
            }
        ),


    Rmv_C1 = 
        Table.RemoveColumns(
            Rnm_C3,
            {"Column33"}
            ),


    Replaced_Value1 = 
        Table.ReplaceValue(
            Rmv_C1,".",",",
            Replacer.ReplaceText,
            {"Column34"}
            ),


    AlterType3 = 
        Table.TransformColumnTypes(
            Replaced_Value1,
            {
                {"Column34", Currency.Type}
            }
        ),


    Rnm_C4 = 
        Table.RenameColumns(
            AlterType3,
            {
                {"Column34", "Ticket"}, 
                {"Column35", "Situacao"}, 
                {"Column36", "CID"}, 
                {"Column37", "CPF_Medico"}, 
                {"Column38", "Nome_Medico"}, 
                {"Column30", "Data_Solicitacao"}
            }
        ),


    Rmv_C2 = 
        Table.RemoveColumns(
            Rnm_C4,
            {
                "Column32"
            }
        ),


    AlterType4 = 
        Table.TransformColumnTypes(
            Rmv_C2,
            {
                {"Idade", Int64.Type}
            }
        ),


    Add_Cond_C = 
        Table.AddColumn(
            AlterType4, 
            "Faixa Etária", 
            each if [Idade] < 12 then "Criança" 
            else if [Idade] < 20 then "Jovem" 
            else if [Idade] < 60 then "Adulto" 
            else "Idoso"
            ),

            
    Doub_C = 
        Table.DuplicateColumn(
            Add_Cond_C, 
            "Cod_Unificado", 
            "Cod_Unificado - Copiar"
            ),


    Rnm_C5 = 
        Table.RenameColumns(
            Doub_C,
            {
                {"Cod_Unificado - Copiar", "Grupo"}
            }
        ),


    Doub_C1 = 
        Table.DuplicateColumn(
            Rnm_C5, "Grupo", "Grupo - Copiar"
            ),


    Rnm_C6 = 
        Table.RenameColumns(
            Doub_C1,
            {
                {"Grupo - Copiar", 
                "Subgrupo - 06"}
            }
        ),


    Ext_Carac1 = 
        Table.TransformColumns(
            Rnm_C6, 
            {
                {"Subgrupo - 06", 
                each Text.Start(_, 6), 
                type text}
            }
        ),


    Rnm_C7 = 
        Table.RenameColumns(
            Ext_Carac1,
            {
                {"Grupo", "Subgrupo - 04"}
            }
        ),


    Ext_Carac2 = 
        Table.TransformColumns(
            Rnm_C7, 
            {
                {"Subgrupo - 04", 
                each Text.Start(_, 4), 
                type text}
            }
        ),


    Doub_C2 = 
        Table.DuplicateColumn(
            Ext_Carac2, 
            "Cod_Unificado", 
            "Cod_Unificado - Copiar"
            ),


    Rnm_C8 = 
        Table.RenameColumns(
            Doub_C2,
            {
                {"Cod_Unificado - Copiar", "Grupo"}
            }
        ),


    Ext_Carac3 = 
        Table.TransformColumns(
            Rnm_C8, 
            {
                {"Grupo", 
                each Text.Start(_, 2), 
                type text}
            }
        ),

        
    Reord_C = 
        Table.ReorderColumns(
            Ext_Carac3,
            {
                "Executante", 
                "Mês", "Ano", 
                "Cod_Solicitacao", 
                "Cod_Interno", 
                "Cod_Unificado", 
                "Procedimento", 
                "CPF", 
                "Profissional", 
                "Data_Execução", 
                "Horario", 
                "Logic", 
                "CNS/SUS", 
                "Paciente", 
                "Data_Nasc", 
                "Idade", 
                "Lote_Paciente", 
                "Nome_Mae", 
                "Endereço", 
                "Complemento1", 
                "Complemento_2", 
                "Numero", 
                "Bairro", 
                "CEP", 
                "Telefone", 
                "Municipio", 
                "Cod_IBGE", 
                "Dominio", 
                "CNES", 
                "Solicitante", 
                "Sexo", 
                "Data_Solicitacao", 
                "Marcador_SISREG", 
                "Ticket", 
                "Situacao", 
                "CID", 
                "CPF_Medico", 
                "Nome_Medico", 
                "Faixa Etária", 
                "Grupo", 
                "Subgrupo - 04", 
                "Subgrupo - 06"
            }
        ),


    Rnm_C9 = 
        Table.RenameColumns(
            Reord_C,
            {
                {"Subgrupo - 04", "SUB-04"}, 
                {"Subgrupo - 06", "SUB-06"}, 
                {"Grupo", "Grupo"}
            }
        ),


    Add_Cond_C1 = 
        Table.AddColumn(
            Rnm_C9, 
            "Expurgo", each if [Sexo] = "F" 
            then "Não" else if [Sexo] = "M" 
            then "Não" else "Sim"),


    Rnm_C10 = 
        Table.RenameColumns(
            Add_Cond_C1,
            {
                {"Expurgo", "Erro Sisreg"}
            }
        )

in
    Rnm_C10